## Deepseek 的配置

In [1]:
from openai import OpenAI
from dotenv import load_dotenv

import os

load_dotenv()

test_api_key = os.getenv("DEEPSEEK_API_KEY")
test_url = 'https://api.deepseek.com/beta'

client = OpenAI(api_key=test_api_key, base_url= test_url)


## system prompt

通用的设置包含一下几个方面

1. 系统的角色和任务
2. 回答过程中的基本要求
3. 回答格式的通用要求

In [2]:

# 定义为高中数学老师，并添加性格特点描述和口吻描述
# 定义回答的风格和格式要求

role_sys = "你是一个富有耐心，和蔼可亲的高中数学老师，任务是回答学生提出的数学问题。"

scope_def = """
            回答问题依据的知识范围：
            1. 知识范围：请以高中数学知识为基础，确保回答内容适合高中学生理解；
            2. 问题范围：仅回答与数学相关的问题。如果学生提问数学无关的问题，请礼貌拒绝；
"""

style_req = """
            回答风格要求：
            3. 步骤化的回答：将回答分为清晰的步骤，每一步包含三个方面，第一方面是依据什么信号执行什么操作，第二方面是计算和推导的详细过程，第三方面是每一步得到的结果，并且用 box 展示出来；
            4. 术语和符号要求：不要使用高中学生不熟悉的名词和符号；
            6. 计算和化简方面要求：必须要有详细的计算整理过程，坚决不允许跳过任何中间步骤；
"""

format_req = """
            回答格式要求：
            7. 结果表示：结果一定要化为最简形式，如果结果是无理数，要进行分母有理化，结果不可以用无限小数表示；
            8. 数学公式：使用 Latex 展示数学公式，行内数学公式使用 $，居中数学公式使用 $$ 表示，一定不要使用 \( 和 \) 或 \[ 和 \]；
            9. 公式过长时，一定要分行展示，分行时，符号要放在行首；
            不推荐：
            $$
            S_n - T_n = (1 + 2 + 3 + \cdots + n) - (2^1 + 2^2 + 2^3 + \cdots + 2^n) 
            $$
            推荐：
            $$
            \begin{aligned}
            S_n - T_n & = (1 + 2 + 3 + \cdots + n) \\
            & - (2^1 + 2^2 + 2^3 + \cdots + 2^n) 
            \end{aligned}
            $$
            10. 对齐要求：多个居中的公式堆叠在一起时，一定要放到同一个 $$...$$ 中，并用 aligned 进行对齐；
            推荐：
            $$
            \begin{aligned}
            a^2 & = 1 - b^2 \\
            & = 1 - (\dfrac{1}{3})^2 \\
            & = 1 - \dfrac{1}{9} \\
            & = \dfrac{8}{9}
            \end{aligned}
            $$
            不推荐：
            $$a^2 = 1- b^2$$
            $$a^2 = 1 - (\dfrac{1}{3})^2$$
            $$a^2 = 1 - \dfrac{1}{9}$$
            $$a^2 = \dfrac{8}{9}$$
            11. 符号要求：使用 \geqslant 表示 \geq，\leqslant 表示\leq，\dfrac 表示 \frac，不要使用 \hline；
            12. 求和操作一定不能用 \sum 符号表示，请用展开式来表示，
            示例：
            不推荐：$\sum_{k=1}^{n} k$,
            推荐：$1+2+3+...+n$。
"""


# 定义长度
lenght_control = ""



## 生成解答

In [71]:
# 问题
problem = "在 $\triangle ABC$ 中，若 $B = \dfrac{\pi}{3}$，求 $\sin A + \sin C$ 的取值范围"

# 解题方法
solving_method = "解决三角形的问题的大致思路是把 $\sin A + \sin C$ 化为角 $A$ 的表达式，然后求范围。"

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": f"""
            {role_sys}。
            {scope_def}。
            {style_req}。
            {format_req}。
            {solving_method}。
            {lenght_control}。
        """}, 
        {"role": "user", "content": f"{problem}"},
    ],
    temperature=0,
    stream=False,
)

print(response.choices[0].message.content)

在三角形 $ABC$ 中，已知角 $B = \dfrac{\pi}{3}$，我们需要求 $\sin A + \sin C$ 的取值范围。我们可以按照以下步骤来解决这个问题：

**步骤 1：利用三角形内角和公式**

根据三角形内角和公式，我们有：
$$
A + B + C = \pi
$$
已知 $B = \dfrac{\pi}{3}$，所以：
$$
A + C = \pi - B = \pi - \dfrac{\pi}{3} = \dfrac{2\pi}{3}
$$
因此，$C = \dfrac{2\pi}{3} - A$。

**步骤 2：将 $\sin A + \sin C$ 表达为角 $A$ 的函数**

利用正弦函数的和角公式，我们有：
$$
\sin A + \sin C = \sin A + \sin\left(\dfrac{2\pi}{3} - A\right)
$$
利用正弦差角公式：
$$
\sin\left(\dfrac{2\pi}{3} - A\right) = \sin\dfrac{2\pi}{3} \cos A - \cos\dfrac{2\pi}{3} \sin A
$$
已知 $\sin\dfrac{2\pi}{3} = \dfrac{\sqrt{3}}{2}$ 和 $\cos\dfrac{2\pi}{3} = -\dfrac{1}{2}$，所以：
$$
\sin\left(\dfrac{2\pi}{3} - A\right) = \dfrac{\sqrt{3}}{2} \cos A - \left(-\dfrac{1}{2}\right) \sin A = \dfrac{\sqrt{3}}{2} \cos A + \dfrac{1}{2} \sin A
$$
因此：
$$
\sin A + \sin C = \sin A + \dfrac{\sqrt{3}}{2} \cos A + \dfrac{1}{2} \sin A = \dfrac{3}{2} \sin A + \dfrac{\sqrt{3}}{2} \cos A
$$

**步骤 3：将表达式化简为单一正弦函数的形式**

我们可以将 $\dfrac{3}{2} \sin A + \dfrac{\sqrt{3}}{2} \cos A$ 表示为 $R \sin(A + \a

## 生成 summary

In [ ]:
summary_gen = """
            请根据学生提出的问题和得到的回答，写出知识点总结，要求：
                1. 对每个步骤的知识点进行总结，包括基于什么信号想到的，起到了什么作用
                2. 列举知识点的常见的使用情景，并举出简单的例子；
                3. 将生成的内容放入 <summary> 的标签中。
"""

if response.choices[0].message.content:
    summary_response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": f"""
                {role_sys}。
                {scope_def}。
                {format_req}。
                学生提出的问题是：{problem}。
                学生得到的回答是：{response.choices[0].message.content}。
                {summary_gen}。
            """},
        ],
        temperature=0,
        stream=False, 
    )
    print("\n ================== summary ====================== \n")
    print(summary_response.choices[0].message.content)
    print("\n ================== summary ====================== \n")




 ================== summary ====================== 


<summary>
**知识点总结：**

1. **三角形内角和公式**  
   - **信号**：题目中给出了角 $B$ 的值，并且需要求其他角的关系。  
   - **作用**：利用三角形内角和公式 $A + B + C = \pi$，可以将角 $C$ 表示为 $\dfrac{2\pi}{3} - A$，从而将问题转化为关于角 $A$ 的函数。  
   - **常见使用情景**：在已知一个角的情况下，求其他角的关系。例如，已知三角形的一个角为 $\dfrac{\pi}{4}$，求另外两个角的和。  
   - **例子**：在三角形 $ABC$ 中，已知角 $A = \dfrac{\pi}{4}$，则角 $B + C = \dfrac{3\pi}{4}$。

2. **正弦函数的和角公式**  
   - **信号**：需要将 $\sin A + \sin C$ 表达为角 $A$ 的函数。  
   - **作用**：利用正弦差角公式 $\sin\left(\dfrac{2\pi}{3} - A\right) = \sin\dfrac{2\pi}{3} \cos A - \cos\dfrac{2\pi}{3} \sin A$，将 $\sin A + \sin C$ 转化为 $\dfrac{3}{2} \sin A + \dfrac{\sqrt{3}}{2} \cos A$。  
   - **常见使用情景**：在三角函数的化简中，将多个正弦或余弦函数合并为一个单一的三角函数。  
   - **例子**：化简 $\sin x + \sin\left(\dfrac{\pi}{3} - x\right)$。

3. **将表达式化简为单一正弦函数的形式**  
   - **信号**：需要将 $\dfrac{3}{2} \sin A + \dfrac{\sqrt{3}}{2} \cos A$ 化简为 $R \sin(A + \alpha)$ 的形式。  
   - **作用**：通过计算 $R$ 和 $\alpha$，将表达式化简为 $\sqrt{3} \sin\left(A + \dfrac{\pi}{6}\right)$，便于后续分析。  
 

## 生成 followup problem

In [ ]:
# 生成followup问题

followup_gen = """
           请根据学生提出的问题，生成一个相似的新问题，并回答，要求：
              1. 新问题和学生所问的问题类似，只是数值或者条件有所改变； 
              2. 新问题必须具备合理性，不可以出现无解的情况；
              3. 数值设计要合理，不可以出现过于复杂的计算；
              4. 新的回答所用的方法和步骤与原回答相似；
              5. 将生成的内容放入 <followup> 的标签中。 
"""

if response.choices[0].message.content:
    follow_response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": f"""
            {role_sys}。
            {format_req}。
            学生提出的问题是：{problem}。
            学生得到的回答是：{response.choices[0].message.content}。
            {followup_gen}。
        """},
    ],
    temperature=0,
    stream=False,
    )
    print("\n ================== followup ====================== \n")
    print(follow_response.choices[0].message.content)
    print("\n ================== followup ====================== \n")



 ================== followup ====================== 


<followup>
**新问题：** 在 $\triangle ABC$ 中，若 $B = \dfrac{\pi}{4}$，求 $\sin A + \sin C$ 的取值范围。

**解答：**

**步骤 1：利用三角形内角和公式**

根据三角形内角和公式，我们有：
$$
A + B + C = \pi
$$
已知 $B = \dfrac{\pi}{4}$，所以：
$$
A + C = \pi - B = \pi - \dfrac{\pi}{4} = \dfrac{3\pi}{4}
$$
因此，$C = \dfrac{3\pi}{4} - A$。

**步骤 2：将 $\sin A + \sin C$ 表达为角 $A$ 的函数**

利用正弦函数的和角公式，我们有：
$$
\sin A + \sin C = \sin A + \sin\left(\dfrac{3\pi}{4} - A\right)
$$
利用正弦差角公式：
$$
\sin\left(\dfrac{3\pi}{4} - A\right) = \sin\dfrac{3\pi}{4} \cos A - \cos\dfrac{3\pi}{4} \sin A
$$
我们知道：
$$
\sin\dfrac{3\pi}{4} = \dfrac{\sqrt{2}}{2}, \quad \cos\dfrac{3\pi}{4} = -\dfrac{\sqrt{2}}{2}
$$
因此：
$$
\sin\left(\dfrac{3\pi}{4} - A\right) = \dfrac{\sqrt{2}}{2} \cos A - \left(-\dfrac{\sqrt{2}}{2}\right) \sin A = \dfrac{\sqrt{2}}{2} \cos A + \dfrac{\sqrt{2}}{2} \sin A
$$
所以：
$$
\sin A + \sin C = \sin A + \dfrac{\sqrt{2}}{2} \cos A + \dfrac{\sqrt{2}}{2} \sin A = \left(1 + \dfrac{\sqrt{2}}{2}\right) \sin A + \d

## 定时 call function

In [76]:
import threading
import time
import sys
import json

# 提供帮助
help_info = """
            询问学生是否需要帮助，要求：
                1. 语气要温柔，不要给学生施加压力；
                2. 不需要前缀，直接询问学生的需求；
                3. 需要调用查询用户答题结果的函数
"""


user_info = [
    {"id": "hang", "result": "错误", "wrong_option": "$(1, 2)$"},
    {"id": "albert", "result": "错误", "wrong_option": "$(\sqrt{3}, 2\sqrt{3})$"},
]

func_call = [
    {
        "type": "function", 
        "function": {
            "name": "get_user_info",
            "description": "获得用户的题目的答题结果",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "string",
                        "description": "学生的 id",
                    },
                    "result": {
                        "type": "string",
                        "description": "学生的答题结果",
                    },
                    "wrong_option":{
                        "type": "string",
                        "description": "学生的错误选项",
                    }
                },
            },
        }
    }
        
]

def get_user_info(user_data):
    user_id = user_data.get("id")  # 从输入数据中提取 id
    if not user_id:
        return None  # 如果 id 不存在，返回 None
    
    # 遍历 user_info，查找匹配的用户
    for user in user_info:
        if user["id"] == user_id:
            return user  # 返回匹配的用户信息
    return None  # 如果未找到用户，返回 None

# 模拟调用 DeepSeek 模型的函数
def call_deepseek_model():
    print("====================================")
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": f"""
                {role_sys}。
                {help_info}。
                学生的 id 是 hang
            """}, 
        ],
        tools=func_call,
        temperature=0,
        stream=False,
    )

    if response.choices[0].finish_reason == 'tool_calls':
        if response.choices[0].message.tool_calls[0].function.name == "get_user_info":
            id = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
            result = get_user_info(id).get('result')
            wrong_option = get_user_info(id).get('wrong_option')
            final_response = client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                    {"role": "system", "content": f"""
                        {role_sys}。
                        学生解决的问题是：{problem}。
                        学生的答题结果是：{result}。
                        学生的错误选项是：{wrong_option}。
                        请给学生提出合理的改正建议。
                    """}, 
                ],
                temperature=0,
                stream=False,
            )
            print('===========')
            print(final_response.choices[0].message.content)
    
    print('回答结束')
    sys.exit(0)


# 设置定时任务
def schedule_function(delay):
    print(f"定时任务已设置，将在 {delay} 秒后启动...")
    timer = threading.Timer(delay, call_deepseek_model)
    timer.start()

# 主程序
if __name__ == "__main__":
    delay = 1  # 10 秒后启动
    schedule_function(delay)

    # 主线程继续运行
    for i in range(1, delay + 1):
        print(f"等待中... {i} 秒")
        time.sleep(1)


定时任务已设置，将在 1 秒后启动...
等待中... 1 秒


首先，我们需要理解题目所给的条件和要求。题目中给出了一个三角形 \( ABC \)，其中角 \( B = \frac{\pi}{3} \)（即 60 度），要求我们求 \( \sin A + \sin C \) 的取值范围。

### 第一步：分析已知条件

1. **三角形内角和**：在任何三角形中，三个内角的和为 \( \pi \) 弧度（即 180 度）。因此，我们有：
   \[
   A + B + C = \pi
   \]
   已知 \( B = \frac{\pi}{3} \)，所以：
   \[
   A + C = \pi - \frac{\pi}{3} = \frac{2\pi}{3}
   \]

2. **角度的范围**：由于 \( A \) 和 \( C \) 都是三角形的内角，所以它们都必须在 \( (0, \pi) \) 之间。此外，由于 \( A + C = \frac{2\pi}{3} \)，我们可以设 \( A = x \)，那么 \( C = \frac{2\pi}{3} - x \)，其中 \( 0 < x < \frac{2\pi}{3} \)。

### 第二步：表达 \( \sin A + \sin C \)

我们需要求 \( \sin A + \sin C \) 的表达式。根据三角函数的和角公式，我们有：
\[
\sin A + \sin C = 2 \sin \left( \frac{A + C}{2} \right) \cos \left( \frac{A - C}{2} \right)
\]
已知 \( A + C = \frac{2\pi}{3} \)，所以：
\[
\sin A + \sin C = 2 \sin \left( \frac{\frac{2\pi}{3}}{2} \right) \cos \left( \frac{A - C}{2} \right) = 2 \sin \left( \frac{\pi}{3} \right) \cos \left( \frac{A - C}{2} \right)
\]
我们知道 \( \sin \left( \frac{\pi}{3} \right) = \frac{\sqrt{3}}{2} \)，所以：
\[
\sin A + \sin C = 2 \ti

In [23]:
sys.exit(0)

SystemExit: 0